# Multiple Views

In [1]:
pwd

/home/chris/workspace/python-course/Level 4/01 Django


Restart the server:

In [3]:
fuser -k 7000/tcp
python manage.py runserver 7000 &

7000/tcp:            36913
[1] 37286


Now add a new model in `tennis/models.py`:

In [4]:
cat << EOF > tennis/models.py
from django.db import models

class Match(models.Model):
    homeTeam = models.CharField(max_length=20)
    homeScore = models.IntegerField(default=0)
    awayTeam = models.CharField(max_length=20)
    awayScore = models.IntegerField(default=0)

    def play(self, home, score1, away, score2):
        self.homeTeam = home
        self.homeScore = score1
        self.awayTeam = away
        self.awayScore = score2
EOF

Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).

You have 18 unapplied migration(s). Your project may not work properly until you apply the migrations for app(s): admin, auth, contenttypes, sessions.
Run 'python manage.py migrate' to apply them.
November 30, 2024 - 16:21:27
Django version 5.1.3, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.

cat << EOF > tennis/models.py^J


Now we need to create tables in our database corresponding to our new models.  To do that, run the following command:

In [5]:
python manage.py migrate

Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_group_name_max_length... OK
  Applying auth.0011_update_proxy_permissions... OK
  Applying auth.0012_alter_user_first_name_max_l

In [6]:
p manage.py makemigrations
p manage.py migrate

No changes detected
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions
Running migrations:
  No migrations to apply.


Migrations are Django’s way of propagating changes you make to your models (adding a field, deleting a model, etc.) into your database schema.

Note that by default we are using the `sqlite3` database:

In [7]:
cat mysite/settings.py | grep ENGINE

        'ENGINE': 'django.db.backends.sqlite3',


The `migrate` command has now created a new class in `tennis\apps.py` 

In [8]:
cat tennis/apps.py

from django.apps import AppConfig


class TennisConfig(AppConfig):
    default_auto_field = 'django.db.models.BigAutoField'
    name = 'tennis'


This new class has to be published in the global `mysite/settings.py` file.  
This is a large file, but the relevent section is as follows:

In [9]:
sed -n '/^INSTALLED_APPS/,/^]/p' mysite/settings.py

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
]


Now add our `tennis.apps.TennisConfig` class to this list:

In [10]:
sed -i -e "/INSTALLED_APPS/ a \    'tennis.apps.TennisConfig'," mysite/settings.py

The relevant part of this file now reads:

In [11]:
sed -n '/^INSTALLED_APPS/,/^]/p' mysite/settings.py

/home/chris/workspace/python-course/src/38 Django/workspace/mysite/mysite/settings.py changed, reloading.
Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
November 30, 2024 - 16:21:46
Django version 5.1.3, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.

sed -n '/^INSTALLED_APPS/,/^]/p' mysite/settings.py^JINSTALLED_APPS = [
    'tennis.apps.TennisConfig',
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
]


Now tell Django that you’ve made some changes to your models (in this case, you’ve made new ones) and that you’d like the changes to be stored as a migration:

In [12]:
python manage.py makemigrations tennis

Migrations for 'tennis':
  tennis/migrations/0001_initial.py
    + Create model Match


Let's take a moment to see the migrations Django has created:

In [13]:
python manage.py sqlmigrate tennis 0001

BEGIN;
--
-- Create model Match
--
CREATE TABLE "tennis_match" ("id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, "homeTeam" varchar(20) NOT NULL, "homeScore" integer NOT NULL, "awayTeam" varchar(20) NOT NULL, "awayScore" integer NOT NULL);
COMMIT;


Now, run migrate again to create those model tables in your database:

In [14]:
python manage.py migrate &

[2] 37667


Before we use these new classes, let's check our webapp is still working:

In [15]:
firefox http:localhost:7000/tennis &

Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions, tennis
Running migrations:
  Applying tennis.0001_initial... OK
[3] 37725
[2]   Done                    python manage.py migrate


Now return to the project root

In [16]:
cd "$projectRoot"
pwd

[30/Nov/2024 16:22:04] "GET /tennis/ HTTP/1.1" 200 41
Not Found: /favicon.ico
[3]+  Done                    firefox http:localhost:7000/tennis
/home/chris/workspace/python-course/Level 4/01 Django/workspace/mysite
